In [2]:
import pandas as pd
import numpy as np
import sqlite3
import importlib

# keeping company information in additional file
import data_file

In [ ]:
importlib.reload(data_file)

In [134]:
# import sales data
discount_data = pd.read_excel('data_files/discount_source.xlsx')

conn = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"  # Replace 'tablename' with your table name
df_customers = pd.read_sql_query(query, conn)
conn.close()

In [135]:
df_customers['tier'] = df_customers['tier'].fillna(df_customers['indirect_direct'])
df_customers['tier_new'] = df_customers['tier_new'].fillna(df_customers['indirect_direct'])

In [136]:
sales_person_n = data_file.sales_person_n
exlbu=data_file.exlbu
exlpanrter=data_file.exlpanrter

In [137]:
order_data = discount_data.copy()
wdf = df_customers.copy()

In [138]:
len(discount_data)

3968

In [139]:
len(order_data)

3968

In [140]:
len(wdf)

14802

In [141]:
order_columns = list(order_data.columns) + ['customer_name', 'indirect_direct', 'tier', 'tier_new', 'type']

In [142]:
# special case with specific SO for agent
agent_so = data_file.special_so
agent_so_two = data_file.special_so_two 
agent_one = data_file.agent_one
so_order_data = order_data[ (order_data['sales_order_so'] == agent_so) | (order_data['sales_order_so'] == agent_so_two)]

so_order_data['customer_name'] = agent_one
so_order_data['indirect_direct'] = 'Indirect'
so_order_data['tier'] = 'Channel Partner'
so_order_data['type'] = 'Agent'

In [150]:
indirect_list = list(wdf['tier'].unique()).remove('Direct')

TypeError: 'str' object cannot be interpreted as an integer

In [149]:
list(wdf['tier'].unique())

['Independent Sales Company',
 'Direct',
 'Indirect Business Partner',
 'Channel Partner']

In [106]:
# special case with specific cutomers for agent
partners_data = wdf[wdf['tier'].isin(indirect_list) | wdf[exlpanrter].notna()]
agent_two = data_file.agent_two
partners_data.loc[partners_data[exlpanrter].notna(), 'customer_name'] = agent_two
partners_data = partners_data.loc[:, ['sold_to_customer', 'agent_person', 'company_code_n', 'sold_to_customer_n','customer_name', 
       'indirect_direct', 'channel', 'type', 'tier', 'tier_new']]

partners_data['sold_to_customer'] = partners_data['sold_to_customer'].astype(str)

In [107]:
agent_partners = partners_data[partners_data['agent_person'].notna()]

In [108]:
# agent sales records preparation
agents_order_data = order_data[order_data['sales_person_n'].isin(sales_person_n)]
agents_order_data = agents_order_data.merge(agent_partners, left_on='sales_person_n', right_on='sold_to_customer_n', how='left')
agents_order_data = agents_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
agents_order_data.columns = agents_order_data.columns.str.replace('_x', '')
agents_order_data = agents_order_data.loc[:, order_columns]
agents_order_data['type'] = 'Agent'

In [109]:
# exclude agents sales from order data not to double lines
order_data = order_data[~order_data['sales_person_n'].isin(sales_person_n)]

In [110]:
partners_data['sold_to_customer'] = partners_data['sold_to_customer'].astype(str)
order_data['sold_to_customer'] = order_data['sold_to_customer'].astype(str)
order_data['sold_to_customer'] = order_data['sold_to_customer'].str.strip()

# all partners except agents data preparation
other_order_data = order_data[(order_data['sold_to_customer'].isin(partners_data['sold_to_customer'])) |(order_data['sold_to_customer_n_latest'].isin(partners_data['sold_to_customer_n']))]

In [111]:
other_order_data = other_order_data.merge(partners_data, left_on='sold_to_customer', right_on='sold_to_customer', how='left')
other_order_data = other_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
other_order_data.columns = other_order_data.columns.str.replace('_x', '')
other_order_data = other_order_data.loc[:, order_columns]
# Update values in columns based on the specific value
other_order_data.loc[other_order_data['indirect_direct'] == 'Direct', 'tier'] = 'Channel Partner'
other_order_data.loc[other_order_data['indirect_direct'] == 'Direct', 'type'] = 'Agent'

In [130]:
# combining all the sales records
full_data['sold_to_customer'] = full_data['sold_to_customer'].astype(str)
full_data = pd.concat([other_order_data, agents_order_data, so_order_data])
full_data.reset_index(inplace=True, drop=True)

In [131]:
len(full_data)

1986

In [124]:
check_df = discount_data.copy()

check_df['sold_to_customer'] = check_df['sold_to_customer'].astype(str)
check_df['sold_to_customer'] = check_df['sold_to_customer'].str.strip()

check_df = check_df[(~check_df['sold_to_customer'].isin(full_data['sold_to_customer']))|(~check_df['sold_to_customer_n_latest'].isin(full_data['sold_to_customer_n_latest']))]

In [125]:
check = list(check_df['sold_to_customer_n_latest'].unique())

In [126]:
writeit = discount_data[discount_data['sold_to_customer_n_latest'].isin(check)]

In [132]:
writer = pd.ExcelWriter('data_files/writeit.xlsx')
writeit.to_excel(writer, sheet_name='writeit', index=False)
full_data.to_excel(writer, sheet_name='222', index=False)
writer.close()

In [123]:
full_data

,company_code_n,sales_order_so,so_item,sstp_approval_no,sold_to_customer,sales_person_n,sold_to_customer_n_latest,ec_eu_customer_n,bu,bu_n,...,sales_eur,discount %,cogs(ms)_eur,gp_eur,Gross Margin,customer_name,indirect_direct,tier,tier_new,type
0,YEF-A,2.008720e+09,30,VYDA0013-0012-000,200239154,Böck Hans-Jürgen,Axiom angewandte Prozesstechnik GmbH,Axiom angewandte Prozesstechnik GmbH,YY116,Transmitters,...,4965,0.7233,3227,1738,0.3500,AXIOM ANGEWANDTE PROZESSTECHNIK,Direct,Channel Partner,Direct,Agent
1,YEF-A,2.008720e+09,40,NaN,200239154,Böck Hans-Jürgen,Axiom angewandte Prozesstechnik GmbH,Axiom angewandte Prozesstechnik GmbH,YY116,Transmitters,...,8084,NaN,4446,3638,0.4500,AXIOM ANGEWANDTE PROZESSTECHNIK,Direct,Channel Partner,Direct,Agent
2,YEF-A,2.008720e+09,41,NaN,200239154,Böck Hans-Jürgen,Axiom angewandte Prozesstechnik GmbH,Axiom angewandte Prozesstechnik GmbH,YY116,Transmitters,...,1401,NaN,771,630,0.4500,AXIOM ANGEWANDTE PROZESSTECHNIK,Direct,Channel Partner,Direct,Agent
3,YEF-A,2.008720e+09,42,NaN,200239154,Böck Hans-Jürgen,Axiom angewandte Prozesstechnik GmbH,Axiom angewandte Prozesstechnik GmbH,YY116,Transmitters,...,1401,NaN,771,630,0.4500,AXIOM ANGEWANDTE PROZESSTECHNIK,Direct,Channel Partner,Direct,Agent
4,YEF-A,2.008720e+09,43,NaN,200239154,Böck Hans-Jürgen,Axiom angewandte Prozesstechnik GmbH,Axiom angewandte Prozesstechnik GmbH,YY116,Transmitters,...,1277,NaN,703,575,0.4500,AXIOM ANGEWANDTE PROZESSTECHNIK,Direct,Channel Partner,Direct,Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,YEF-I,2.008743e+09,20,NaN,200275006,Baglieri Maurizio,ANDRITZ NOVIMPIANTI SRL,ABC Tissue Products Pty Ltd,YY116,Transmitters,...,744,0.5927,470,274,0.3682,BAGLIERI MAURIZIO,Indirect,Channel Partner,Channel Partner,Agent
1982,YEF-I,2.008804e+09,10,NaN,200344811,AUMICO SNC PCI Locatelli Alberto & Andre,CAFFARO GREEN CHEMICALS SRL,CAFFARO GREEN CHEMICALS SRL,YY116,Transmitters,...,1017,0.4543,487,530,0.5211,AUMICO,Indirect,Channel Partner,Channel Partner,Agent
1983,YEF-I,2.008804e+09,20,NaN,200344811,AUMICO SNC PCI Locatelli Alberto & Andre,CAFFARO GREEN CHEMICALS SRL,CAFFARO GREEN CHEMICALS SRL,YY116,Transmitters,...,540,0.3713,346,195,0.3604,AUMICO,Indirect,Channel Partner,Channel Partner,Agent
1984,YEF-I,2.008915e+09,10,NaN,200274755,Baglieri Maurizio,BIRLA CARBON ITALY SRL,BIRLA CARBON ITALY SRL,YY116,Transmitters,...,351,0.0028,185,166,0.4717,BAGLIERI MAURIZIO,Indirect,Channel Partner,Channel Partner,Agent
